In [1]:
#!/usr/bin/env python

import os
import numpy as np
import pandas as pd
from datetime import datetime
import warnings

warnings.filterwarnings("ignore")

# ============================================================
# CONFIG
# ============================================================

UNIVERSE_FILE   = "./12-tradable_sp500_universe/12-tradable_sp500_universe.parquet"
SPY_REGIME_FILE = "./8-SPY_200DMA_market_regime/8-SPY_200DMA_regime.parquet"
ATR20_DIR       = "./4-ATR20_adjusted_All_Prices"

OUTPUT_DIR      = "./18-risk_report_output"  
os.makedirs(OUTPUT_DIR, exist_ok=True)

START_TRADING   = pd.Timestamp("1999-01-01")

# Match live engine capital / params
INITIAL_CAPITAL       = 1_000_000
TOP_PERCENTILE        = 0.90      # top 10% by slope_adj
REBALANCE_DAY         = "Wednesday"
TRADING_DAYS_PER_YEAR = 252

# Turnover / trade throttling – keep in sync with live engine
DRIFT_THRESHOLD          = 0.01      # min abs weight diff to rebalance (~1%)
MIN_TRADE_VALUE          = 3_000.0   # skip trades smaller than this
MIN_NEW_POSITION_WEIGHT  = 0.005     # don't open new positions < 0.5% of equity

# Lookahead audit scenarios
LAG_SCENARIOS       = [0, 1, 3, 5]   # deterministic lags (in trading days)
RAND_LAG_MAX        = 3              # random lag between 0 and RAND_LAG_MAX (inclusive)

# Debug ticker (optional; set to None to disable)
DEBUG_TICKER = None  # e.g. "TWX"


# ============================================================
# FAST PRICE LOOKUP
# ============================================================

def fast_price_lookup(px_array, date_val):
    """
    Given a structured array for a ticker:
        px_array['date'] (datetime64[ns])
        px_array['px']   (float)
    return the latest price <= date_val.
    """
    date_val = np.datetime64(date_val, "ns")
    dates = px_array["date"]
    idx = np.searchsorted(dates, date_val, side="right") - 1
    if idx < 0:
        return np.nan
    return px_array["px"][idx]


# ============================================================
# LOAD UNIVERSE
# ============================================================

print("Loading universe…")
df = pd.read_parquet(UNIVERSE_FILE)
df["date"] = pd.to_datetime(df["date"])

df["slope_adj"] = pd.to_numeric(df["slope_adj"], errors="coerce")
df["close_adj"] = pd.to_numeric(df["close_adj"], errors="coerce")

print(f"Loaded universe: {len(df):,} rows")

# ============================================================
# LOAD SPY REGIME AND MERGE
# ============================================================

print("Loading SPY regime file…")
spy = pd.read_parquet(SPY_REGIME_FILE)

# Ensure we have a 'date' column
if spy.index.name in ["Date", "date", None]:
    spy = spy.reset_index().rename(columns={"index": "date", "Date": "date"})

spy["date"] = pd.to_datetime(spy["date"])

# Expect columns: spy_close, spy_ma200, market_regime (0/1)
required_cols = ["spy_close", "spy_ma200", "market_regime"]
missing = [c for c in required_cols if c not in spy.columns]
if missing:
    raise ValueError(f"SPY regime file missing columns: {missing}")

# Merge SPY info into universe for each date
df = df.merge(
    spy[["date", "spy_close", "spy_ma200", "market_regime"]],
    on="date",
    how="left"
)

# ============================================================
# MERGE ATR20 PER-TICKER
# ============================================================

print("Merging ATR20 per ticker…")
atr20_map = {}
for f in os.listdir(ATR20_DIR):
    if not f.endswith(".parquet"):
        continue
    t = f.replace(".parquet", "")
    tmp = pd.read_parquet(os.path.join(ATR20_DIR, f))
    if "atr20" not in tmp:
        continue
    tmp["date"] = pd.to_datetime(tmp["date"])
    atr20_map[t] = tmp[["date", "atr20"]]

rows = []
for t, sub in df.groupby("ticker", sort=False):
    if t in atr20_map:
        rows.append(sub.merge(atr20_map[t], on="date", how="left"))
    else:
        sub = sub.copy()
        sub["atr20"] = np.nan
        rows.append(sub)

df = pd.concat(rows, ignore_index=True)
print(f"Universe with ATR20 merged: {len(df):,} rows")

# ============================================================
# PREP GROUPED DATA
# ============================================================

df_by_date = {d: sub for d, sub in df.groupby("date")}

px_by_ticker = {}
for t, sub in df.groupby("ticker", sort=False):
    sub = sub.sort_values("date")
    arr = np.zeros(
        len(sub),
        dtype=[("date", "datetime64[ns]"), ("px", "float64")]
    )
    arr["date"] = sub["date"].values.astype("datetime64[ns]")
    arr["px"]   = sub["close_adj"].astype(float).values
    px_by_ticker[t] = arr

dates = sorted(df_by_date.keys())
date_idx = {d: i for i, d in enumerate(dates)}  # for quick lag lookups

# ============================================================
# METRIC HELPERS
# ============================================================

def cagr_from_curve(eq: pd.Series):
    eq = eq.dropna()
    if len(eq) < 2 or eq.iloc[0] <= 0:
        return np.nan
    years = (eq.index[-1] - eq.index[0]).days / 365.25
    if years <= 0:
        return np.nan
    return (eq.iloc[-1] / eq.iloc[0]) ** (1.0 / years) - 1.0


def max_dd(series: pd.Series):
    series = series.dropna()
    if series.empty:
        return np.nan
    rollmax = series.cummax()
    dd = series / rollmax - 1.0
    return float(dd.min())

def sharpe_from_curve(series: pd.Series):
    r = series.pct_change().dropna()
    if len(r) < 2 or r.std(ddof=1) == 0:
        return np.nan
    return float(r.mean() / r.std(ddof=1) * np.sqrt(TRADING_DAYS_PER_YEAR))


def sortino_from_curve(series: pd.Series):
    r = series.pct_change().dropna()
    downside = r[r < 0]
    if len(downside) < 2 or downside.std(ddof=1) == 0:
        return np.nan
    return float(r.mean() / downside.std(ddof=1) * np.sqrt(TRADING_DAYS_PER_YEAR))


def show_stats(label, eq: pd.Series, trades: pd.DataFrame):
    c = cagr_from_curve(eq)
    dd = max_dd(eq)
    sh = sharpe_from_curve(eq)
    so = sortino_from_curve(eq)

    # Turnover stats
    if len(eq) > 1:
        years = (eq.index[-1] - eq.index[0]).days / 365.25
    else:
        years = np.nan
    total_traded_val = trades["value"].abs().sum() if not trades.empty else 0.0
    avg_equity = eq.mean() if not eq.empty else np.nan
    turnover_per_year = ((total_traded_val / 2.0) / avg_equity) / years if (years and avg_equity) else np.nan


    n_trades = len(trades)
    n_buys   = (trades["type"] == "BUY").sum()
    n_sells  = (trades["type"] == "SELL").sum()
    avg_trade_val = trades["value"].abs().mean() if n_trades > 0 else np.nan

    print(f"{label}:")
    print(f"  CAGR        : {c*100:6.2f}%")
    print(f"  Max DD      : {dd*100:6.2f}%")
    print(f"  Sharpe      : {sh:6.2f}")
    print(f"  Sortino     : {so:6.2f}")
    print(f"  Trades      : {n_trades} (Buys={n_buys}, Sells={n_sells})")
    print(f"  Avg Trade $ : {avg_trade_val:,.0f}")
    print(f"  Turnover/yr : {turnover_per_year:6.2f}x\n")

    stats = {
        "label": label,
        "CAGR": c,
        "MaxDD": dd,
        "Sharpe": sh,
        "Sortino": so,
        "Trades": n_trades,
        "Buys": n_buys,
        "Sells": n_sells,
        "AvgTradeValue": avg_trade_val,
        "TurnoverPerYear": turnover_per_year,
    }

    # Buy-rank diagnostics if we recorded signal ranks
    if "signal_rank_within_top" in trades.columns:
        buys = trades[trades["type"] == "BUY"]
        if not buys.empty:
            stats["AvgBuyRank"] = buys["signal_rank_within_top"].mean()
            stats["MedBuyRank"] = buys["signal_rank_within_top"].median()
            stats["BuyRankP90"] = buys["signal_rank_within_top"].quantile(0.9)
        else:
            stats["AvgBuyRank"] = np.nan
            stats["MedBuyRank"] = np.nan
            stats["BuyRankP90"] = np.nan
    else:
        stats["AvgBuyRank"] = np.nan
        stats["MedBuyRank"] = np.nan
        stats["BuyRankP90"] = np.nan

    return stats


# ============================================================
# UTILITIES
# ============================================================

def is_rebalance_day(date):
    """Weekly rebalance on a specific weekday (e.g. Wednesday)."""
    return date.day_name() == REBALANCE_DAY


# ============================================================
# TRADING ENGINE WITH SIGNAL LAG (MATCHES LIVE ENGINE LOGIC)
# ============================================================

def run_strategy_with_lag(
    signal_lag_days=0,
    random_lag=False,
    max_random_lag=RAND_LAG_MAX,
    label=None
):
    """
    Run the weekly regression-only / vol-based / SPY-filter strategy,
    but with **signals computed on an earlier date**.

    For every rebalance date D:
        - Choose signal_date S:
            * deterministic: S = D - signal_lag_days (by trading days)
            * random:        S = D - U(0, max_random_lag)
        - Use S's universe to:
            * rank by slope_adj
            * choose top percentile group
            * use S's atr20 & close_adj for sizing
            * use S's market_regime for SPY filter
        - Execute trades at **D's prices**.
        - Drift thresholds, min trade size, and min new position weight
          are applied using D's prices and portfolio value,
          exactly like the live engine.
    """

    if label is None:
        if random_lag:
            label = f"rand_lag_0_{max_random_lag}"
        else:
            label = f"lag_{signal_lag_days}d"

    print(f"\n=== RUNNING LOOKAHEAD AUDIT: {label} ===")

    cash = INITIAL_CAPITAL
    positions = {}   # ticker -> {"shares": int, "entry": float}
    history = []
    equity_curve = []
    rebal_summaries = []

    last_equity = INITIAL_CAPITAL

    for date in dates:
        if date < START_TRADING:
            continue

        day_prices = df_by_date.get(date)
        if day_prices is None or day_prices.empty:
            continue

        # --------------------------------------------------------
        # DETERMINE SIGNAL DATE FOR THIS REBALANCE
        # --------------------------------------------------------
        signal_day = None
        signal_tg = None
        spy_regime_signal = None  # 1/bull, 0/bear or None

        do_rebalance = is_rebalance_day(date)
        if do_rebalance:
            idx = date_idx[date]
            if random_lag:
                lag = np.random.randint(0, max_random_lag + 1)
            else:
                lag = signal_lag_days

            signal_idx = idx - lag
            if signal_idx >= 0:
                signal_date = dates[signal_idx]
                signal_day = df_by_date.get(signal_date)

        # --------------------------------------------------------
        # REBALANCE USING SIGNALS FROM signal_day
        # --------------------------------------------------------
        if do_rebalance and (signal_day is not None) and (not signal_day.empty):

            univ = signal_day.copy()

            # SPY regime at signal date
            if "market_regime" in univ:
                spy_regime_signal = int(univ["market_regime"].iloc[0])
            else:
                spy_regime_signal = None

            if spy_regime_signal is None or pd.isna(spy_regime_signal):
                # Conservative default: allow buys (like live engine default)
                spy_above_200 = True
            else:
                spy_above_200 = (spy_regime_signal > 0)

            # Rankable universe by slope_adj at signal date
            rankable = univ[univ["slope_adj"].notna()].copy()
            rankable = rankable.sort_values("slope_adj", ascending=False)

            if len(rankable) > 0:
                cutoff = rankable["slope_adj"].quantile(TOP_PERCENTILE)
                top_group = rankable[rankable["slope_adj"] >= cutoff].copy()
            else:
                top_group = rankable.iloc[0:0].copy()

            # Assign ranks within top group
            if not top_group.empty:
                top_group = top_group.sort_values("slope_adj", ascending=False)
                top_group["slope_rank_within_top"] = np.arange(1, len(top_group) + 1)
                top_tickers = set(top_group["ticker"].values)
                rank_map = dict(zip(top_group["ticker"], top_group["slope_rank_within_top"]))
            else:
                top_tickers = set()
                rank_map = {}

            # Filter for valid ATR and price (on signal date)
            tg = top_group[
                top_group["atr20"].notna() &
                (top_group["atr20"] > 0) &
                top_group["close_adj"].notna() &
                (top_group["close_adj"] > 0)
            ].copy()

            if not tg.empty:
                inv_vol = 1.0 / tg["atr20"].astype(float)
                total_inv_vol = inv_vol.sum()
            else:
                inv_vol = None
                total_inv_vol = 0.0

            if total_inv_vol > 0:
                tg = tg.assign(inv_vol=inv_vol)
                # Use last_equity for sizing same as live engine
                tg["target_value_signal"] = last_equity * tg["inv_vol"] / total_inv_vol
                # Sizing based on signal-date close_adj
                tg["target_shares_signal"] = np.floor(
                    tg["target_value_signal"] / tg["close_adj"]
                ).astype(int)
                tg = tg[tg["target_shares_signal"] > 0]
            else:
                tg = tg.iloc[0:0].copy()

            # Quick lookup for signal-day sizing
            signal_sizing = {
                row["ticker"]: row
                for _, row in tg.iterrows()
            }

            # ------------------------
            # SELL LOGIC
            # ------------------------
            sells = []

            for t, pos in list(positions.items()):
                # If a name drops OUT of the signal top group -> sell all
                if t not in top_tickers:
                    sells.append(t)

            for t in sells:
                px_exec = fast_price_lookup(px_by_ticker[t], date)
                value = float(positions[t]["shares"] * px_exec) if not np.isnan(px_exec) else 0.0
                cash += value

                history.append({
                    "date": date,
                    "ticker": t,
                    "type": "SELL",
                    "shares": positions[t]["shares"],
                    "price": float(px_exec),
                    "value": value,
                    "reason": "not_in_signal_top_group",
                    "signal_lag_days": lag if random_lag else signal_lag_days,
                    "signal_date": signal_day["date"].iloc[0],
                    "spy_above_200dma_signal": spy_above_200,
                    "signal_slope_adj": np.nan,
                    "signal_rank_within_top": np.nan,
                })

                del positions[t]

            # ------------------------
            # BUY / REBALANCE LOGIC
            # ------------------------
            # Only consider buys / rebalancing if SPY is bullish at signal date
            if spy_above_200 and not tg.empty:

                # Compute portfolio value *as of execution date* to get weights
                total_portfolio_value = cash
                for t, pos in positions.items():
                    px_today = fast_price_lookup(px_by_ticker[t], date)
                    if not np.isnan(px_today):
                        total_portfolio_value += pos["shares"] * px_today

                if total_portfolio_value <= 0:
                    total_portfolio_value = last_equity

                # Loop over each signal-day top ticker, apply drift / min-size rules
                for ticker, row_sig in signal_sizing.items():
                    # Signal-day sizing
                    target_sh_signal = int(row_sig["target_shares_signal"])
                    signal_price = float(row_sig["close_adj"])
                    signal_slope = float(row_sig["slope_adj"])
                    signal_rank = int(row_sig["slope_rank_within_top"])

                    # Execution price today
                    px_today = fast_price_lookup(px_by_ticker[ticker], date)
                    if np.isnan(px_today):
                        continue

                    # Value of target position at today's prices
                    target_value_today = target_sh_signal * px_today
                    target_weight = target_value_today / total_portfolio_value

                    current_sh = positions.get(ticker, {}).get("shares", 0)
                    current_value_today = current_sh * px_today
                    current_weight = current_value_today / total_portfolio_value if total_portfolio_value > 0 else 0.0

                    weight_diff = abs(target_weight - current_weight)

                    # --- Drift threshold ---
                    if weight_diff < DRIFT_THRESHOLD:
                        continue

                    # --- For NEW positions, enforce min weight ---
                    if current_sh == 0 and target_weight < MIN_NEW_POSITION_WEIGHT:
                        continue

                    # Determine direction
                    if target_sh_signal > current_sh:
                        # BUY
                        diff_sh = target_sh_signal - current_sh
                        trade_value = diff_sh * px_today

                        # --- Min trade size ---
                        if abs(trade_value) < MIN_TRADE_VALUE:
                            continue

                        if trade_value > cash:
                            continue

                        cash -= trade_value
                        positions[ticker] = {
                            "shares": current_sh + diff_sh,
                            "entry": px_today if current_sh == 0 else positions[ticker]["entry"],
                        }

                        history.append({
                            "date": date,
                            "ticker": ticker,
                            "type": "BUY",
                            "shares": diff_sh,
                            "price": float(px_today),
                            "value": float(trade_value),
                            "reason": "rebalance_up" if current_sh > 0 else "new_entry",
                            "signal_lag_days": lag if random_lag else signal_lag_days,
                            "signal_date": signal_day["date"].iloc[0],
                            "spy_above_200dma_signal": spy_above_200,
                            "signal_slope_adj": signal_slope,
                            "signal_rank_within_top": signal_rank,
                        })

                    elif target_sh_signal < current_sh:
                        # SELL (partial)
                        diff_sh = current_sh - target_sh_signal
                        trade_value = diff_sh * px_today

                        # --- Min trade size ---
                        if abs(trade_value) < MIN_TRADE_VALUE:
                            continue

                        cash += trade_value
                        new_sh = current_sh - diff_sh

                        if new_sh <= 0:
                            positions.pop(ticker, None)
                        else:
                            positions[ticker]["shares"] = new_sh

                        history.append({
                            "date": date,
                            "ticker": ticker,
                            "type": "SELL",
                            "shares": diff_sh,
                            "price": float(px_today),
                            "value": float(trade_value),
                            "reason": "rebalance_down",
                            "signal_lag_days": lag if random_lag else signal_lag_days,
                            "signal_date": signal_day["date"].iloc[0],
                            "spy_above_200dma_signal": spy_above_200,
                            "signal_slope_adj": signal_slope,
                            "signal_rank_within_top": signal_rank,
                        })

            # ------------------------------------------------
            # REBALANCE-DAY POSITION / WEIGHT SUMMARY
            # ------------------------------------------------
            # Use post-trade positions & today's prices
            total_value_today = cash
            weights = []
            for t, pos in positions.items():
                px_today = fast_price_lookup(px_by_ticker[t], date)
                if not np.isnan(px_today):
                    total_value_today += pos["shares"] * px_today

            if total_value_today > 0:
                for t, pos in positions.items():
                    px_today = fast_price_lookup(px_by_ticker[t], date)
                    if not np.isnan(px_today):
                        w = (pos["shares"] * px_today) / total_value_today
                        weights.append(w)

            rebal_summaries.append({
                "date": date,
                "label": label,
                "num_positions": len(positions),
                "spy_above_200dma_signal": spy_above_200,
                "avg_weight": float(np.mean(weights)) if weights else np.nan,
                "max_weight": float(np.max(weights)) if weights else np.nan,
                "min_weight": float(np.min(weights)) if weights else np.nan,
                "top5_weight_sum": float(np.sum(sorted(weights, reverse=True)[:5])) if len(weights) >= 1 else np.nan,
            })

        # --------------------------------------------------------
        # DAILY MARK-TO-MARKET
        # --------------------------------------------------------
        equity_val = 0.0
        for t, pos in positions.items():
            px = fast_price_lookup(px_by_ticker[t], date)
            if np.isnan(px):
                continue
            equity_val += pos["shares"] * px

        portfolio_value = cash + equity_val
        last_equity = portfolio_value

        equity_curve.append({
            "date": date,
            "portfolio_value": portfolio_value,
            "cash": cash,
            "num_positions": len(positions),
            "label": label,
        })

    trades = pd.DataFrame(history)
    eq_df  = pd.DataFrame(equity_curve)
    rebal_df = pd.DataFrame(rebal_summaries)

    # Save outputs
    trades_path = os.path.join(OUTPUT_DIR, f"trades_{label}.parquet")
    eq_path     = os.path.join(OUTPUT_DIR, f"equity_{label}.parquet")
    rebal_path  = os.path.join(OUTPUT_DIR, f"rebal_summary_{label}.parquet")

    trades.to_parquet(trades_path, index=False)
    eq_df.to_parquet(eq_path, index=False)
    rebal_df.to_parquet(rebal_path, index=False)

    print(f"Saved trades       → {trades_path}")
    print(f"Saved equity curve → {eq_path}")
    print(f"Saved rebalance    → {rebal_path}")

    # Stats
    eq_series = eq_df.set_index("date")["portfolio_value"]
    stats = show_stats(label, eq_series, trades)

    # Rebalance stats: average positions, max weight, etc.
    if not rebal_df.empty:
        stats["AvgPositions"] = rebal_df["num_positions"].mean()
        stats["MedPositions"] = rebal_df["num_positions"].median()
        stats["AvgMaxWeight"] = rebal_df["max_weight"].mean()
        stats["AvgTop5WeightSum"] = rebal_df["top5_weight_sum"].mean()
    else:
        stats["AvgPositions"] = np.nan
        stats["MedPositions"] = np.nan
        stats["AvgMaxWeight"] = np.nan
        stats["AvgTop5WeightSum"] = np.nan

    return eq_df, trades, rebal_df, stats


# ============================================================
# MAIN: RUN LOOKAHEAD AUDIT SCENARIOS
# ============================================================

def main():
    print("\n=== LOOKAHEAD AUDIT FOR REGRESSION+SPYFILTER SYSTEM ===")
    print(f"Start trading: {START_TRADING.date()}")
    print(f"Initial capital: ${INITIAL_CAPITAL:,.2f}")
    print(f"Top percentile (slope_adj): {TOP_PERCENTILE:.2f}")
    print(f"Rebalance day: {REBALANCE_DAY}")
    print(f"Lag scenarios: {LAG_SCENARIOS}, random lag 0–{RAND_LAG_MAX}\n")

    all_stats = []

    # 1) Deterministic lag scenarios: 0, 1, 3, 5 days
    for lag in LAG_SCENARIOS:
        eq_df, trades, rebal_df, st = run_strategy_with_lag(
            signal_lag_days=lag,
            random_lag=False,
            label=f"lag_{lag}d"
        )
        all_stats.append(st)

    # 2) Random lag scenario: 0–RAND_LAG_MAX days
    eq_rand, trades_rand, rebal_rand, st_rand = run_strategy_with_lag(
        random_lag=True,
        max_random_lag=RAND_LAG_MAX,
        label=f"rand_lag_0_{RAND_LAG_MAX}"
    )
    all_stats.append(st_rand)

    # Summary table
    stats_df = pd.DataFrame(all_stats)
    stats_df["CAGR_%"]   = stats_df["CAGR"]   * 100.0
    stats_df["MaxDD_%"]  = stats_df["MaxDD"]  * 100.0

    cols_order = [
        "label",
        "CAGR_%", "MaxDD_%", "Sharpe", "Sortino",
        "Trades", "Buys", "Sells", "AvgTradeValue", "TurnoverPerYear",
        "AvgBuyRank", "MedBuyRank", "BuyRankP90",
        "AvgPositions", "MedPositions",
        "AvgMaxWeight", "AvgTop5WeightSum",
    ]
    # Only keep columns that exist
    cols_order = [c for c in cols_order if c in stats_df.columns]
    stats_df = stats_df[cols_order].sort_values("CAGR_%", ascending=False)

    print("\n=== LOOKAHEAD AUDIT SUMMARY ===")
    print(stats_df.to_string(index=False, float_format=lambda x: f"{x:6.2f}"))

    summary_path = os.path.join(OUTPUT_DIR, "lookahead_audit_stats_regression_spyfilter.csv")
    stats_df.to_csv(summary_path, index=False)
    print(f"\nSaved audit stats → {summary_path}")
    print("\n=== LOOKAHEAD AUDIT COMPLETE ===\n")


if __name__ == "__main__":
    main()


Loading universe…
Loaded universe: 3,585,390 rows
Loading SPY regime file…
Merging ATR20 per ticker…
Universe with ATR20 merged: 3,585,390 rows

=== LOOKAHEAD AUDIT FOR REGRESSION+SPYFILTER SYSTEM ===
Start trading: 1999-01-01
Initial capital: $1,000,000.00
Top percentile (slope_adj): 0.90
Rebalance day: Wednesday
Lag scenarios: [0, 1, 3, 5], random lag 0–3


=== RUNNING LOOKAHEAD AUDIT: lag_0d ===
Saved trades       → ./18-risk_report_output\trades_lag_0d.parquet
Saved equity curve → ./18-risk_report_output\equity_lag_0d.parquet
Saved rebalance    → ./18-risk_report_output\rebal_summary_lag_0d.parquet
lag_0d:
  CAGR        :  17.78%
  Max DD      : -29.55%
  Sharpe      :   1.06
  Sortino     :   1.32
  Trades      : 12760 (Buys=6262, Sells=6498)
  Avg Trade $ : 565,217
  Turnover/yr :   6.22x


=== RUNNING LOOKAHEAD AUDIT: lag_1d ===
Saved trades       → ./18-risk_report_output\trades_lag_1d.parquet
Saved equity curve → ./18-risk_report_output\equity_lag_1d.parquet
Saved rebalance   